In [2]:
from dataclasses import dataclass
from IPython.display import display, HTML

## Load data

In [3]:
@dataclass
class Cell:
    value: int
    is_drawn: bool = False

        
def load_numbers(stream, sep=None):
    line = stream.readline().strip()
    return [int(x.strip()) for x in line.split(sep)]


def load_board(stream):
    board = {}
    for y in range(5):
        for x, value in enumerate(load_numbers(stream)):
            board[x,y] = Cell(value)
    return board


def load_data(filename):
    # print(f'load_data({filename!r})')
    with open(filename) as f:
        numbers = load_numbers(f, sep=",")
        boards = []
        line = f.readline()
        while line:
            new_board = load_board(f)
            boards.append(new_board)
            line = f.readline()
    return numbers, boards

In [4]:
def is_bingo(board):
    for y in range(5):
        if all(board[x, y].is_drawn for x in range(5)):
            return True
    for x in range(5):
        if all(board[x, y].is_drawn for y in range(5)):
            return True       
    return False

In [5]:
def print_board(board):
    buff = ['<table>']
    for y in range(5):
        buff.append('<tr>')
        for x in range(5):
            buff.append("<td>")  
            value = board[x, y].value
            is_drawn = board[x, y].is_drawn
            buff.append(f'<b>{value}<b>' if is_drawn else f'{value}');
            buff.append("</td>")  
        buff.append('</tr>')
    buff.append('</table><br>')
    display(HTML('\n'.join(buff)))

In [6]:
numbers, boards = load_data('04-sample.txt')
assert len(numbers) == 27
assert len(boards) == 3

In [8]:
for board in boards:
    assert is_bingo(board) is False
    
test_board = {
    (0, 0): Cell(1, is_drawn=True),
    (0, 1): Cell(2),
    (0, 2): Cell(3),
    (0, 3): Cell(4),
    (0, 4): Cell(5),
    (1, 0): Cell(6, is_drawn=True),
    (1, 1): Cell(7),
    (1, 2): Cell(8),
    (1, 3): Cell(9),
    (1, 4): Cell(10),
    (2, 0): Cell(11, is_drawn=True),
    (2, 1): Cell(12),
    (2, 2): Cell(13),
    (2, 3): Cell(14),
    (2, 4): Cell(15),
    (3, 0): Cell(16, is_drawn=True),
    (3, 1): Cell(17),
    (3, 2): Cell(18),
    (3, 3): Cell(19),
    (3, 4): Cell(20),
    (4, 0): Cell(21),
    (4, 1): Cell(22),
    (4, 2): Cell(23),
    (4, 3): Cell(24),
    (4, 4): Cell(25),    
}
assert is_bingo(test_board) is False
print_board(test_board)

1,6,11,16,21
2,7,12,17,22
3,8,13,18,23
4,9,14,19,24
5,10,15,20,25


In [9]:
test_board[4, 0].is_drawn = True
assert is_bingo(test_board) is True
print_board(test_board)

1,6,11,16,21
2,7,12,17,22
3,8,13,18,23
4,9,14,19,24
5,10,15,20,25


In [10]:
def mark_board_number(board, number):
    for y in range(5):
        for x in range(5):
            if board[x, y].value == number:
                board[x, y].is_drawn = True
                return

The score of the winning board can now be calculated. Start by finding the sum of all unmarked numbers on that board; in this case, the sum is 188. Then, multiply that sum by the number that was just called when the board won, 24, to get the final score, 188 * 24 = 4512.

In [11]:
def score_board(board):
    acc = 0
    for y in range(5):
        for x in range(5):
            if not board[x, y].is_drawn:
                acc += board[x, y].value
    return acc

In [12]:
numbers, boards = load_data('04-sample.txt')

In [13]:
def play(filename):
    numbers, boards = load_data(filename)
    for number in numbers:
        # print('drawns', number)
        for board in boards:
            mark_board_number(board, number)
            if is_bingo(board): 
                print_board(board)
                return score_board(board), number
        
assert play('04-sample.txt') == (188, 24)

14,21,17,24,4
10,16,15,9,19
18,8,23,26,20
22,11,13,6,5
2,0,12,3,7


In [14]:
score, num = play('04-input.txt')
print(f'First Solution: {score * num}')

55,15,85,39,4
95,83,27,46,45
19,47,61,9,66
82,32,72,77,16
50,96,14,60,35


First Solution: 72770


## Second part

In [15]:
def hash_board(board):
    items = []
    for y in range(5):
        for x in range(5):
            items.append(str(board[x, y].value))
    return '/'.join(items)

assert hash_board(test_board) == '1/6/11/16/21/2/7/12/17/22/3/8/13/18/23/4/9/14/19/24/5/10/15/20/25'


In [16]:
def find_last_board(filename):
    numbers, boards = load_data(filename)
    last_number = 0
    last_score = 0
    last_board = None
    solutions = set([])
    for number in numbers:
        # print('drawns', number)
        for board in boards:
            mark_board_number(board, number)
            if is_bingo(board) and hash_board(board) not in solutions: 
                last_score = score_board(board)
                last_number = number
                solutions.add(hash_board(board))
    return last_score, last_number
        
assert find_last_board('04-sample.txt') == (148, 13)

In [17]:
score, num = find_last_board('04-input.txt')
print(f'Second Solution: {score * num}')

Second Solution: 13912
